In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler

In [ ]:
trainset = pd.read_csv('./train_open.csv')
testset = pd.read_csv('./test_open.csv')

print(trainset)
print(testset)

X_train = trainset.iloc[:, 0:20].values
Y_train = trainset.iloc[:, 20].values

X_test = testset.iloc[:, 0:20].values
Y_test = testset.iloc[:, 20].values

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
Y_train = torch.from_numpy(Y_train).type(torch.LongTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
Y_test = torch.from_numpy(Y_test).type(torch.LongTensor)


      cache-misses-1  node-loads-1  branch-misses-1  branch-load-misses-1  \
0           36218220       2530196         29315596              29316371   
1           34025893       2310966         29030928              29031111   
2           36778380       2597194         32242092              32242655   
3           27574965       2081723         18248848              18249034   
4           35736215       2499164         32035503              32038530   
...              ...           ...              ...                   ...   
1545        32765499       2249790         28748154              28751282   
1546        36032964       2675011         23717334              23717871   
1547        34176184       2447644         23153779              23154422   
1548        32203221       2189984         28521743              28523470   
1549        34253061       2361245         30368723              30370088   

      cache-misses-2  node-loads-2  branch-misses-2  branch-load-misses-2  

In [ ]:
batch_size = 256
train_set = Data.TensorDataset(X_train, Y_train)
train_loader = Data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(20, 48)
        self.fc2 = nn.Linear(48, 96)
        self.fc3 = nn.Linear(96, 128)
        self.fc4 = nn.Linear(128, 31)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(self.fc4(x))
        
        return x
    
model = MLP()

In [ ]:
num_epochs = 5000
learning_rate = 1e-3
batch_no = len(X_train) // batch_size

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(1, num_epochs+1):
    loss_sum = 0
    for step, (x, y) in enumerate(train_loader):
        y_pred = model(x)
        y = y.squeeze()
        loss = loss_function(y_pred, y)
        loss_sum += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print("epoch: %d, loss: %f" % (epoch, loss_sum/batch_size))
        acc_sum = 0
        acc_sum += (model(X_train).argmax(dim=1) == Y_train.squeeze()).sum()
        print("train accuracy: %f" % (acc_sum/len(Y_train)))

epoch: 50, loss: 0.026867
train accuracy: 0.740000
epoch: 100, loss: 0.018779
train accuracy: 0.779355
epoch: 150, loss: 0.016925
train accuracy: 0.802581
epoch: 200, loss: 0.013197
train accuracy: 0.850968
epoch: 250, loss: 0.010303
train accuracy: 0.864516
epoch: 300, loss: 0.012773
train accuracy: 0.859355
epoch: 350, loss: 0.010843
train accuracy: 0.881290
epoch: 400, loss: 0.008273
train accuracy: 0.881935
epoch: 450, loss: 0.007460
train accuracy: 0.903226
epoch: 500, loss: 0.006823
train accuracy: 0.906452
epoch: 550, loss: 0.008396
train accuracy: 0.909032
epoch: 600, loss: 0.006360
train accuracy: 0.910323
epoch: 650, loss: 0.005765
train accuracy: 0.920000
epoch: 700, loss: 0.004978
train accuracy: 0.922581
epoch: 750, loss: 0.004699
train accuracy: 0.936129
epoch: 800, loss: 0.004893
train accuracy: 0.925161
epoch: 850, loss: 0.005239
train accuracy: 0.920645
epoch: 900, loss: 0.003759
train accuracy: 0.932903
epoch: 950, loss: 0.004596
train accuracy: 0.919355
epoch: 1000, 

In [ ]:
model.eval()
acc_sum = 0
print(model(X_test).argmax(dim=1))
print(Y_test.squeeze())
acc_sum += (model(X_test).argmax(dim=1) == Y_test.squeeze()).sum()
print("test accuracy: %f" % (acc_sum/len(Y_test)))

tensor([ 3,  9,  6, 20, 30,  8, 23, 13, 29, 30, 12, 25, 18, 22, 26, 30, 17, 18,
        19, 24, 23, 20, 23, 10, 13,  3,  2,  1,  2,  1,  5, 19, 17, 23,  4,  3,
        12, 25, 10, 15, 30,  6,  7,  9, 10, 22,  6, 19, 27, 15, 11,  8, 12, 26,
        24,  8, 28, 17,  6,  7,  1,  3,  9, 27, 18,  4,  6, 18, 19,  3, 23, 10,
         5, 27, 18, 29, 18, 20, 29, 28,  0, 17,  6, 18, 15, 19,  4, 15, 16,  8,
        23,  9, 11, 21,  4, 14, 15, 21,  1, 30, 16,  9,  3, 15, 14, 13, 19, 21,
        14, 12, 16,  6,  6, 22,  9, 29, 25, 11, 22, 26, 15, 20, 12, 30,  7, 29,
         6, 27, 15, 21, 12, 18, 29, 23, 17, 24,  6,  4,  5,  1,  2,  4, 17, 12,
         2,  6, 18, 13, 27, 29, 30, 12, 19, 21, 11, 12,  8,  4, 24, 10,  2,  0,
        28, 28, 14, 15, 24, 20, 24,  5, 15, 19, 12,  4, 17, 25, 13, 25, 11, 15,
        26, 19,  3, 25, 14, 15,  5,  8,  8, 14,  2, 14, 23, 23, 20,  6,  1,  8,
         7, 23, 16, 30, 14, 21, 11, 28, 30, 29, 20, 17,  6,  9,  1, 11, 30, 10,
         1, 29, 10,  7,  1, 14, 16, 24, 

In [ ]:
batch_size = 256
train_set = Data.TensorDataset(X_train, Y_train)
train_loader = Data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=True
)

class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc1 = nn.LSTM(20, 48)
        self.fc2 = nn.Linear(48, 96)
        self.fc3 = nn.Linear(96, 31)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x, _ = self.fc1(x.view(len(x), 1, -1))
        x = self.fc2(x.view(len(x), 48))
        x = F.relu(x)
        x = self.dropout(self.fc3(x))
        
        return x
    
model = LSTM()

In [ ]:
num_epochs = 5000
learning_rate = 1e-3
batch_no = len(X_train) // batch_size

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(1, num_epochs+1):
    loss_sum = 0
    for step, (x, y) in enumerate(train_loader):
        y_pred = model(x)
        y = y.squeeze()
        loss = loss_function(y_pred, y)
        loss_sum += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print("epoch: %d, loss: %f" % (epoch, loss_sum/batch_size))
        acc_sum = 0
        acc_sum += (model(X_train).argmax(dim=1) == Y_train.squeeze()).sum()
        print("train accuracy: %f" % (acc_sum/len(Y_train)))

epoch: 50, loss: 0.034463
train accuracy: 0.685806
epoch: 100, loss: 0.023148
train accuracy: 0.745806
epoch: 150, loss: 0.022472
train accuracy: 0.785806
epoch: 200, loss: 0.018702
train accuracy: 0.807097
epoch: 250, loss: 0.016048
train accuracy: 0.807742
epoch: 300, loss: 0.016648
train accuracy: 0.823226
epoch: 350, loss: 0.013059
train accuracy: 0.832258
epoch: 400, loss: 0.011482
train accuracy: 0.850968
epoch: 450, loss: 0.011480
train accuracy: 0.862581
epoch: 500, loss: 0.010533
train accuracy: 0.863226
epoch: 550, loss: 0.011325
train accuracy: 0.858065
epoch: 600, loss: 0.009258
train accuracy: 0.875484
epoch: 650, loss: 0.010269
train accuracy: 0.872903
epoch: 700, loss: 0.007596
train accuracy: 0.890968
epoch: 750, loss: 0.008104
train accuracy: 0.905161
epoch: 800, loss: 0.008029
train accuracy: 0.897419
epoch: 850, loss: 0.007929
train accuracy: 0.901290
epoch: 900, loss: 0.006231
train accuracy: 0.897419
epoch: 950, loss: 0.007057
train accuracy: 0.909677
epoch: 1000, 

In [ ]:
model.eval()
acc_sum = 0
print(model(X_test).argmax(dim=1))
print(Y_test.squeeze())
acc_sum += (model(X_test.view(len(X_test), 1, -1)).argmax(dim=1) == Y_test.squeeze()).sum()
print("test accuracy: %f" % (acc_sum/len(Y_test)))

tensor([ 3,  9,  0, 20, 30,  8, 23, 13, 29, 13, 19, 25, 18, 24, 21, 30, 13, 18,
        19, 24, 23, 20, 23, 10, 13,  3,  2,  1,  2,  1,  5, 19, 17, 23,  4,  3,
        12, 25, 10, 15, 30,  6, 30,  9, 10, 22,  6, 19, 27, 15, 11,  8, 12, 26,
        24,  8, 28, 17,  6,  7,  1,  3,  9, 27, 28,  4,  6, 18, 19,  3, 23, 10,
         5, 27, 18, 29, 18, 20, 25, 28,  0, 17,  6, 18, 15, 19,  4, 15, 16, 24,
        23,  9, 15, 21,  4, 14,  2, 21,  1, 30, 16,  9,  3, 15, 14, 13, 19, 21,
        14, 12, 16,  6,  0, 17,  9, 29, 25, 11, 11,  2, 15, 20, 12, 30,  7, 29,
         6, 27, 15, 21, 12, 18, 29, 23, 17, 24,  6,  4,  5,  1,  2,  4, 17, 12,
         0,  6,  7, 13, 27, 29, 17, 12, 19, 27, 11, 12,  8,  4, 24, 10,  2,  0,
        21, 28, 14, 15, 24, 20, 22,  5, 15, 19, 12,  4, 17, 25, 13, 25, 11, 15,
        29, 19,  3, 25, 14, 15, 19, 16, 24, 14,  0, 14, 23, 23, 20,  5,  1,  8,
         7, 23, 16, 30, 14, 21, 11, 28, 30, 29, 20, 17,  6, 28,  1, 11, 30, 10,
         1, 29, 10,  0,  1, 14, 16, 24, 

In [ ]:
trainset = pd.read_csv('./train_open.csv')
testset = pd.read_csv('./test_open.csv')

print(trainset)
print(testset)

X_train = trainset.iloc[:, 0:20].values
Y_train = trainset.iloc[:, 20].values

X_test = testset.iloc[:, 0:20].values
Y_test = testset.iloc[:, 20].values

X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
Y_train = torch.from_numpy(Y_train).type(torch.LongTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
Y_test = torch.from_numpy(Y_test).type(torch.LongTensor)


      cache-misses-1  node-loads-1  branch-misses-1  branch-load-misses-1  \
0           36218220       2530196         29315596              29316371   
1           34025893       2310966         29030928              29031111   
2           36778380       2597194         32242092              32242655   
3           27574965       2081723         18248848              18249034   
4           35736215       2499164         32035503              32038530   
...              ...           ...              ...                   ...   
1545        32765499       2249790         28748154              28751282   
1546        36032964       2675011         23717334              23717871   
1547        34176184       2447644         23153779              23154422   
1548        32203221       2189984         28521743              28523470   
1549        34253061       2361245         30368723              30370088   

      cache-misses-2  node-loads-2  branch-misses-2  branch-load-misses-2  

In [ ]:
batch_size = 512
train_set = Data.TensorDataset(X_train, Y_train)
train_loader = Data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=True
)
model = MLP()

num_epochs = 5000
learning_rate = 1e-4
batch_no = len(X_train) // batch_size

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(1, num_epochs+1):
    loss_sum = 0
    for step, (x, y) in enumerate(train_loader):
        y_pred = model(x)
        y = y.squeeze()
        loss = loss_function(y_pred, y)
        loss_sum += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print("epoch: %d, loss: %f" % (epoch, loss_sum/batch_size))
        acc_sum = 0
        acc_sum += (model(X_train).argmax(dim=1) == Y_train.squeeze()).sum()
        print("train accuracy: %f" % (acc_sum/len(Y_train)))

epoch: 50, loss: 959.498779
train accuracy: 0.185806
epoch: 100, loss: 436.590515
train accuracy: 0.344516
epoch: 150, loss: 254.286652
train accuracy: 0.409032
epoch: 200, loss: 248.000259
train accuracy: 0.418065
epoch: 250, loss: 148.978638
train accuracy: 0.484516
epoch: 300, loss: 140.730072
train accuracy: 0.519355
epoch: 350, loss: 120.782654
train accuracy: 0.525806
epoch: 400, loss: 91.838745
train accuracy: 0.550968
epoch: 450, loss: 89.856956
train accuracy: 0.591613
epoch: 500, loss: 139.464264
train accuracy: 0.556774
epoch: 550, loss: 73.121948
train accuracy: 0.567742
epoch: 600, loss: 72.751381
train accuracy: 0.592903
epoch: 650, loss: 91.708054
train accuracy: 0.578065
epoch: 700, loss: 83.268364
train accuracy: 0.664516
epoch: 750, loss: 90.647385
train accuracy: 0.620645
epoch: 800, loss: 65.534286
train accuracy: 0.629677
epoch: 850, loss: 56.716789
train accuracy: 0.633548
epoch: 900, loss: 67.574806
train accuracy: 0.622581
epoch: 950, loss: 58.306499
train accur

In [ ]:
model.eval()
acc_sum = 0
print(model(X_test).argmax(dim=1))
print(Y_test.squeeze())
acc_sum += (model(X_test).argmax(dim=1) == Y_test.squeeze()).sum()
print("test accuracy: %f" % (acc_sum/len(Y_test)))

tensor([ 3,  9,  6, 20, 25,  8, 23, 13, 29, 30, 12, 25, 26, 22, 13, 30, 13, 28,
        19, 24, 25, 20, 23, 10, 13,  3,  2,  1,  2,  1,  5, 19, 17, 23,  4,  3,
        12, 25, 10, 15, 30,  6, 30, 29, 10, 22,  6, 19, 27, 15, 28, 28, 12, 26,
        24,  8, 28, 17,  6,  7,  1,  3, 11, 22, 28,  4,  6, 18, 19,  3, 23, 10,
         5, 13, 18, 29, 28, 20, 15, 28,  0, 17,  4, 18,  2, 19,  4, 15, 16, 28,
        23, 29, 15, 21,  4, 14,  2, 21,  1, 30, 16, 29,  3, 15, 12, 13,  7, 21,
        28, 16, 16,  6,  6, 22,  9, 29, 25, 11, 22, 21, 29, 20, 12,  7,  7, 29,
         6, 27, 15, 21, 13, 18, 29, 23, 17, 25,  6,  4,  5,  1,  2,  4, 17, 12,
         0,  6,  7, 13, 27, 29, 22, 12, 19, 21, 11, 12,  8,  4, 24, 10,  2,  0,
        21, 28, 14, 15,  5, 20, 25,  5, 15, 19, 12,  4, 17, 29, 13, 25, 11, 15,
        29, 19,  3, 25, 21, 15,  7, 16, 24, 28,  4, 14, 23, 23, 20,  6,  1,  8,
         7, 23, 16, 22, 14, 21, 19, 28,  5,  9, 20, 17,  6, 28,  1, 11, 30, 10,
         1, 29, 10,  7,  1,  7, 16, 24, 